<table align="center">
  <td align="center"><a target="_blank" href="http://introtodeeplearning.com">
        <img src="http://introtodeeplearning.com/images/colab/mit.png" style="padding-bottom:5px;" />
      Visit MIT Deep Learning</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/drive/1oiQvklyWy_dkSEuowvIUdmZE3jzb4B7X">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/jariasf/MIT-6.S191/blob/master/lab2/Part2_debiasing.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>


# Laboratory 2: Computer Vision

# Part 2: Debiasing Facial Detection Systems

In the second portion of the lab, we'll explore two prominent aspects of applied deep learning: facial detection and algorithmic bias. 

Deploying fair, unbiased AI systems is critical to their long-term acceptance. Consider the task of facial detection: given an image, is it an image of a face? [Recent work from the MIT Media Lab](http://proceedings.mlr.press/v81/buolamwini18a/buolamwini18a.pdf) showed that this seemingly simple, but extremely important, task is subject to extreme amounts of algorithmic bias among select demographics. [Another report](https://ieeexplore.ieee.org/document/6327355) analyzed the face detection system used by the US law enforcement and found that it had significantly lower accuracy among dark skinned women between the age of 18-30 years old. 

Run the next code block for a short video from Google that explores how and why it's important to consider bias when thinking about machine learning:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('59bMh59JQDo')

In this lab, we'll investigate [one recently published approach](http://introtodeeplearning.com/AIES_2019_Algorithmic_Bias.pdf) to addressing algorithmic bias. We'll build a facial detection model that learns the *latent variables* underlying face image datasets and uses this to adaptively re-sample the training data, thus mitigating any biases that may be present in order  to train a *debiased* model.

Let's get started by installing the relevant dependencies:

In [ ]:
import tensorflow as tf
# tf.enable_eager_execution()

import functools
import matplotlib.pyplot as plt
import numpy as np
import pdb

# Dependencies
! pip install regex

# Download the class repository
! git clone https://github.com/aamini/introtodeeplearning_labs.git  > /dev/null 2>&1
! cd introtodeeplearning_labs 
! git pull
! cd .. 

# Import the necessary class-specific utility files for this lab
import introtodeeplearning_labs as util

In [ ]:
!pip install mitdeeplearning
import mitdeeplearning as mdl

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import h5py
import sys

IM_SHAPE = (64, 64, 3)
class TrainingDatasetLoader(object):
    def __init__(self, data_path):
        print("Opening {}".format(data_path))
        sys.stdout.flush()

        self.cache = h5py.File(data_path, 'r')

        print("Loading data into memory...")
        sys.stdout.flush()
        self.images = self.cache['images'][:]
        self.labels = self.cache['labels'][:]
        self.image_dims = self.images.shape
        n_train_samples = self.image_dims[0]

        self.train_inds = np.random.permutation(np.arange(n_train_samples))

        self.pos_train_inds = self.train_inds[ self.labels[self.train_inds, 0] == 1.0 ]
        self.neg_train_inds = self.train_inds[ self.labels[self.train_inds, 0] != 1.0 ]

    def get_train_size(self):
        return self.train_inds.shape[0]

    def get_train_steps_per_epoch(self, batch_size, factor=10):
        return self.get_train_size()//factor//batch_size

    def get_batch(self, n, only_faces=False, p_pos=None, p_neg=None, return_inds=False):
        if only_faces:
            selected_inds = np.random.choice(self.pos_train_inds, size=n, replace=False, p=p_pos)
        else:
            selected_pos_inds = np.random.choice(self.pos_train_inds, size=n//2, replace=False, p=p_pos)
            selected_neg_inds = np.random.choice(self.neg_train_inds, size=n//2, replace=False, p=p_neg)
            selected_inds = np.concatenate((selected_pos_inds, selected_neg_inds))

        sorted_inds = np.sort(selected_inds)
        train_img = self.images[sorted_inds,:,:,::-1]/255.
        train_label = self.labels[sorted_inds,...]
        return (train_img, train_label, sorted_inds) if return_inds else (train_img, train_label)

    def get_n_most_prob_faces(self, prob, n):
        idx = np.argsort(prob)[::-1]
        most_prob_inds = self.pos_train_inds[idx[:10*n:10]]
        return self.images[most_prob_inds,...]/255.

    def get_all_train_faces(self):
        return self.images[ self.pos_train_inds ]



class PPBFaceEvaluator:
    ''' Evaluate on the PPB dataset'''
    def __init__(self, skip=1):

        path_to_faces = tf.keras.utils.get_file('ppb', 'https://www.dropbox.com/s/l0lp6qxeplumouf/PPB.tar?dl=1', extract=True)
        self.ppb_root = os.path.join(os.path.split(path_to_faces)[0], 'PPB-2017')

        ppb_anno = os.path.join(self.ppb_root,'PPB-2017-metadata.csv')

        self.anno_dict = {}
        with open(ppb_anno) as f:
#             print(f.read())
            for line in f.readlines()[1:]:
                ind, name, gender, numeric, skin, country = line.split(',')
#                 print(name.lower().strip())
                self.anno_dict[name.strip()] = (gender.lower().strip(),skin.lower().strip())

        image_dir = os.path.join(self.ppb_root, "imgs")
        image_files = sorted(os.listdir(image_dir))[::skip] #sample every 4 images for computation time in the lab

        self.raw_images = {
            'male_darker':[],
            'male_lighter':[],
            'female_darker':[],
            'female_lighter':[],
        }

        for filename in image_files:
            if not filename.endswith(".jpg"):
                continue
            image = cv2.imread(os.path.join(image_dir,filename))[:,:,::-1]
            gender, skin = self.anno_dict[filename]
            self.raw_images[gender+'_'+skin].append(image)


    def get_sample_faces_from_demographic(self, gender, skin_color):
        key = self.__get_key(gender, skin_color)
        data = self.raw_images[key][50]/255.
        return data


    def evaluate(self, models_to_test, gender, skin_color, output_idx=None, from_logit=False, patch_stride=0.2, patch_depth=5):
        correct_predictions = [0.0]*len(models_to_test)

        key = self.__get_key(gender, skin_color)
        num_faces = len(self.raw_images[key])

        import progressbar
        bar = progressbar.ProgressBar()
        for face_idx in bar(range(num_faces)):

            image = self.raw_images[key][face_idx]
            height, width, _ = image.shape

            patches, bboxes = slide_square(image, patch_stride, width/2, width, patch_depth)
            patches = tf.cast(tf.constant(patches, dtype=tf.uint8), tf.float32)/255.

            for model_idx, model in enumerate(models_to_test):
                out = model(patches)
                y = out if output_idx is None else out[output_idx]
                y = y.numpy()
                y_inds = np.argsort(y.flatten())
                most_likely_prob = y[y_inds[-1]]
                if (from_logit and most_likely_prob >= 0.0) or \
                   (not from_logit and most_likely_prob >= 0.5):
                        correct_predictions[model_idx] += 1

        accuracy = [correct_predictions[i]/num_faces for i,_ in enumerate(models_to_test)]
        return accuracy


    def __get_key(self, gender, skin_color):
        gender = gender.lower()
        skin_color = skin_color.lower()
        assert gender in ['male', 'female']
        assert skin_color in ['lighter', 'darker']
        return '{}_{}'.format(gender, skin_color)



''' function to slide a square across image and extract square regions
img = the image
stride = (0,1], provides the fraction of the dimension for which to slide to generate a crop
max_size = maximum square size
min_size = minimum square size
n = number of different sizes including min_size, max_size '''
def slide_square(img, stride, min_size, max_size, n):
    img_h, img_w = img.shape[:2]

    square_sizes = np.linspace(min_size, max_size, n, dtype=np.int32)
    square_images = []
    square_bbox = [] # list of list of tuples: [(i1,j1), (i2,j2)] where i1,j1 is the top left corner; i2,j2 is bottom right corner
    # for each of the square_sizes
    for level, sq_dim in enumerate(square_sizes):

        stride_length = int(stride*sq_dim)
        stride_start_i = range(0,int(img_h-sq_dim+1),stride_length)
        stride_start_j = range(0,int(img_w-sq_dim+1),stride_length)
        for i in stride_start_i:
            for j in stride_start_j:
                square_top_left = (i,j)

                square_bottom_right = (i+sq_dim,j+sq_dim)

                square_corners = (square_top_left,square_bottom_right)
                square_corners_show = ((j,i),(j+sq_dim,i+sq_dim))
                square_image = img[i:i+sq_dim,j:j+sq_dim]

                square_resize = cv2.resize(square_image, IM_SHAPE[:2], interpolation=cv2.INTER_NEAREST)
                # append to list of images and bounding boxes

                square_images.append(square_resize)
                square_bbox.append(square_corners)

    return square_images, square_bbox

## 2.1 Datasets

We'll be using three datasets in this lab. In order to train our facial detection models, we'll need a dataset of positive examples (i.e., of faces) and a dataset of negative examples (i.e., of things that are not faces). We'll use these data to train our models to classify images as either faces or not faces. Finally, we'll need a test dataset of face images. Since we're concerned about the potential *bias* of our learned models against certain demographics, it's important that the test dataset we use has equal representation across the demographics or features of interest. In this lab, we'll consider skin tone and gender. 

1.   **Positive training data**: [CelebA Dataset](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html). A large-scale (over 200K images) of celebrity faces.   
2.   **Negative training data**: [ImageNet](http://www.image-net.org/). Many images across many different categories. We'll take negative examples from a variety of non-human categories. 
3. **Test data**: [Pilot Parliaments Benchmark](http://proceedings.mlr.press/v81/buolamwini18a/buolamwini18a.pdf) (PPB). The PPB dataset consists of images of 1270 male and female parliamentarians from various African and European countries and exhibits parity in both skin tone and gender. The gender of each face is annotated with the sex-based "Male'' and "Female'' labels. Skin tone annotations are based on the Fitzpatrick skin type classification system, with each image labeled as "Lighter'' or "Darker''.

Let's begin by importing these datasets: 

In [ ]:
# Get the training data: both images from CelebA and ImageNet
path_to_training_data = tf.keras.utils.get_file('train_face.h5', 'https://www.dropbox.com/s/l5iqduhe0gwxumq/train_face.h5?dl=1')

This directly downloads the raw data. We've written two classes that do a bit of data pre-processing and import the results in a usable format: `TrainingDatasetLoader` for the training data and `PPBFaceEvaluator` for the test data.

Let's create a `TrainingDatasetLoader` and use it to take a look at the training data.

In [ ]:
# Instantiate a TrainingDatasetLoader using the downloaded dataset
loader = TrainingDatasetLoader(path_to_training_data)

We can look at the size of the training dataset and grab a batch of size 100:

In [ ]:
number_of_training_examples = loader.get_train_size()
(images, labels) = loader.get_batch(100)

Play around with displaying images to get a sense of what the training data actually looks like!

In [ ]:
#@title Change the sliders to look at positive and negative training examples! { run: "auto" }

face_images = images[np.where(labels==1)[0]]
not_face_images = images[np.where(labels==0)[0]]

idx_face = 20 #@param {type:"slider", min:0, max:50, step:1}
idx_not_face = 15 #@param {type:"slider", min:0, max:50, step:1}

plt.figure(figsize=(4,2))
plt.subplot(1, 2, 1)
plt.imshow(face_images[idx_face])
plt.title("Face")
plt.grid(False)

plt.subplot(1, 2, 2)
plt.imshow(not_face_images[idx_not_face])
plt.title("Not Face")
plt.grid(False)

We can also create a `PPBFaceEvaluator` instance for the PPB dataset and display some example images. We'll use this dataset later on in the evaluation step.

In [ ]:
#@title { run: "auto" }

ppb = PPBFaceEvaluator() # create the dataset handler

gender = "male" #@param ["male", "female"]
skin_color = "darker" #@param ["lighter", "darker"]

img = ppb.get_sample_faces_from_demographic(gender, skin_color)
plt.imshow(img)
plt.grid(False)

### Thinking about bias

Remember we'll be training our facial detection classifiers on the large, well-curated CelebA dataset (and ImageNet), and then evaluating their accuracy by testing them on the PPB dataset. Our goal is to build a model that trains on CelebA *and* achieves high classification accuracy on PPB across all demographics, and to thus show that this model does not suffer from any hidden bias. 

What exactly do we mean when we say a classifier is biased? In order to formalize this, we'll need to think about [*latent variables*](https://en.wikipedia.org/wiki/Latent_variable), variables that define a dataset but are not strictly observed. As defined in the generative modeling lecture, we'll use the term *latent space* to refer to the probability distributions of the aforementioned latent variables. Putting these ideas together, we consider a classifier *biased* if its classification decision changes after it sees some additional latent features. This notion of bias may be helpful to keep in mind throughout the rest of the lab. 

## 2.2 CNN for facial detection 

First, we'll define and train a CNN on the facial classification task, and evaluate its accuracy on the PPB dataset. Later, we'll evaluate the performance of our debiased models against this baseline CNN. The CNN model has a relatively standard architecture consisting of a series of convolutional layers with batch normalization followed by two fully connected layers to flatten the convolution output and generate a class prediction. 

### Define and train the CNN model

Like we did in the first part of the lab, we'll define our CNN model, and then train on the CelebA and ImageNet datasets using the `tf.GradientTape` class and the `tf.GradientTape.gradient` method.

In [ ]:
n_outputs = 1 # number of outputs (i.e., face or not face)
n_filters = 12 # base number of convolutional filters

'''Function to define a standard CNN model'''
def make_standard_classifier():
    Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
    BatchNormalization = tf.keras.layers.BatchNormalization
    Flatten = tf.keras.layers.Flatten
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

    model = tf.keras.Sequential([
        Conv2D(filters=1*n_filters, kernel_size=[5,5],  strides=[2,2], input_shape=(64,64,3)),
        BatchNormalization(),
        
        Conv2D(filters=2*n_filters, kernel_size=[5,5],  strides=[2,2]),
        BatchNormalization(),

        Conv2D(filters=4*n_filters, kernel_size=[3,3],  strides=[2,2]),
        BatchNormalization(),

        Conv2D(filters=6*n_filters, kernel_size=[3,3],  strides=[2,2]),
        BatchNormalization(),

        Flatten(),
        Dense(1, activation=None),
        tf.keras.layers.Dropout(0.5)
    ])
    return model
  
standard_classifier = make_standard_classifier()

Now let's train the standard CNN!

**IMPORTANT NOTE: For educational purposes (to keep training times low), in this lab we are not training our models (both this CNN and a second model later on) for very long, and are only using a small, randomly sampled fraction of the CelebA dataset to train. As a result, the gap in accuracies between the standard CNN classifier and the debiased model (which you'll define soon) will not be as pronounced as if you were to train the models for longer and/or with more data.**

In [ ]:
batch_size = 36
num_epochs = 10  # keep small to run faster
learning_rate = 1e-3

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate) # define our optimizer
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) # to record the evolution of the loss
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')

# The training loop!
for epoch in range(num_epochs):
  
#   custom_msg = mdl.util.custom_progress_text("Epoch: %(epoch).0f Loss: %(loss)2.2f")
#   bar = mdl.util.create_progress_bar(custom_msg)
  
  for idx in range(loader.get_train_steps_per_epoch(batch_size)):
    # First grab a batch of training data and convert the input images to tensors
    x, y = loader.get_batch(batch_size)
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    
    # GradientTape to record differentiation operations
    with tf.GradientTape() as tape:
      logits = standard_classifier(x) # feed the images into the model
      loss_value = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits) # compute the loss

#     custom_msg.update_mapping(epoch=epoch, loss=loss_value.numpy().mean())
    # Backpropagation
    grads = tape.gradient(loss_value, standard_classifier.variables)
    optimizer.apply_gradients(zip(grads, standard_classifier.variables))

    loss_history.append(loss_value.numpy().mean()) 
    plotter.plot(loss_history.get())

### Evaluate performance of the standard CNN

Next, let's evaluate the classification performance of our CelebA-trained standard CNN on the training dataset and the PPB dataset. For the PPB data, we'll look at the classification accuracy across four different demographics defined in PPB: dark-skinned male, dark-skinned female, light-skinned male, and light-skinned female.


In [ ]:
# Evaluate on a subset of CelebA+Imagenet
(batch_x, batch_y) = loader.get_batch(5000)
y_pred_standard = tf.round(tf.nn.sigmoid(standard_classifier.predict(batch_x)))
acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))
print("Standard CNN accuracy on (potentially biased) training set: {:.4f}".format(acc_standard.numpy()))

In [ ]:
# Evaluate on PPB dataset (takes ~3 minutes)
standard_cnn_accuracy = []
for skin_color in ['lighter', 'darker']:
    for gender in ['male', 'female']:
        standard_cnn_accuracy.append( ppb.evaluate([standard_classifier], gender, skin_color, from_logit=True)[0] )
        print("{} {}: {}".format(gender, skin_color, standard_cnn_accuracy[-1]))

In [ ]:
plt.bar(range(4), standard_cnn_accuracy)
plt.xticks(range(4), ('LM', 'LF', 'DM', 'DF'))
plt.ylim(np.min(standard_cnn_accuracy)-0.01,np.max(standard_cnn_accuracy)+0.01)
plt.ylabel('Accuracy')

Take a look at the accuracies for this first model across these four groups. What do you observe? Would you consider this model biased or unbiased, and why? 

## 2.3 Variational autoencoder (VAE) for learning latent structure

As you saw, the accuracy of the CNN varies across the four demographics we looked at. To think about why this may be, consider the dataset the model was trained on, CelebA. If certain features, such as dark skin or hats, are *rare* in CelebA, the model may end up biased against these as a result of training with a biased dataset. That is to say, its classification accuracy will be worse on faces that have under-represented features, such as dark-skinned faces or faces with hats, relevative to faces have features that are well-represented in the training data! This is a problem. 

Our goal is to train a *debiased* version of this classifier -- one that accounts for potential disparities in feature representation within the training data. Specifically, to build a debiased facial classifier, we'll train a model that learns a representation of the underlying latent space to the face training data. The model then uses this information to mitigate unwanted biases by sampling faces with rare features, like dark skin or hats, *more frequently* during training. The key design requirement for our model is that it can learn an *encoding* of the latent features in the face data in an entirely *unsupervised* way. To achieve this, we'll turn to variational autoencoders (VAEs).

![The concept of a VAE](http://kvfrans.com/content/images/2016/08/vae.jpg)

As shown in the schematic above, VAEs rely on an encoder-decoder structure to learn a latent representation of the input data. In the context of computer vision, the encoder network takes in input images, encodes them into a series of variables defined by a mean and standard deviation, and then draws from the distributions defined by these parameters to generate a set of sampled latent variables. The decoder network then "decodes" these variables to generate a reconstruction of the original image, which is used during training to help the model identify which latent variables are important to learn. 

Let's formalize two key aspects of the VAE model and define relevant functions for each.


### Understanding VAEs: loss function

In practice, how can we train a VAE? In doing the reparameterization above, we constrain the means and standard deviations to approximately follow a unit Gaussian. Recall that these are learned parameters, and therefore must factor into the loss computation, and that the decoder portion of the VAE is using these parameters to output a reconstruction that should closely match the input image, which also must factor into the loss. What this means is that we'll have two terms in our VAE loss function:

1.  **Latent loss ($L_{KL}$)**: measures how closely the learned latent variables match a unit Gaussian and is defined by the Kullback-Leibler (KL) divergence. Note that the reparameterization trick is what makes this loss function differentiable!
2.   **Reconstruction loss ($L_{x}{(x,\hat{x})}$)**: measures how accurately the reconstructed outputs match the input and is given by the $L^2$ norm of the input image and its reconstructed output.  

The equations for both of these losses are provided below:

$$ L_{KL}(\mu, \sigma) = \frac{1}{2}\sum\limits_{j=0}^{k-1}\small{(\sigma_j^2 + \mu_j^2 - 1 - \log{\sigma_j^2})} $$

$$ L_{x}{(x,\hat{x})} = ||x-\hat{x}||_2 $$ 

Thus for the VAE loss we have: 

$$ L_{VAE} = c\cdot L_{KL} + L_{x}{(x,\hat{x})} $$

where $c$ is a weighting coefficient used for regularization. 

Now we're ready to define our VAE loss function:

In [ ]:
# Function to calculate VAE loss given an input x, reconstructed output x_pred, 
#    encoded means mu, encoded log of standard deviation logsigma, and weight parameter for the latent loss
def vae_loss_function(x, x_pred, mu, logsigma, kl_weight=0.0005):
  '''TODO: Define the latent loss.
  Hint: note that this function takes in logsigma!'''
  latent_loss = 0.5 * tf.reduce_sum(tf.exp(logsigma) + mu**2 - 1 - logsigma, axis=1)
  '''TODO: Define the reconstruction loss. Hint: you'll need to use tf.reduce_mean to compute the mean-squared error, 
  with axis=(1,2,3)'''
  reconstruction_loss = tf.reduce_mean( (x - x_pred)**2, axis=(1,2,3) )
  '''TODO: Define the total VAE loss, i.e., weighted combination of two losses above.'''
  total_vae_loss = kl_weight * latent_loss + reconstruction_loss
  return total_vae_loss

Great! Now that we have a more concrete sense of how VAEs work, let's explore how we can leverage this network structure to train a *debiased* facial classifier.

### Understanding VAEs: reparameterization 

As you may recall from lecture, VAEs use a "reparameterization  trick" for sampling learned latent variables. Instead of the VAE encoder generating a single vector of real numbers for each latent variable, it generates a vector of means and a vector of standard deviations that are constrained to roughly follow Gaussian distributions. We then sample from the standard deviations and add back the mean to output this as our sampled latent vector. Formalizing this for a latent variable $z$ where we sample $\epsilon \sim \mathcal{N}(0,(I))$ we have: 

$$ z = \mathbb{\mu} + e^{\left(\frac{1}{2} \cdot \log{\Sigma}\right)}\circ \epsilon $$

where $\mu$ is the mean and $\Sigma$ is the covariance matrix. This is useful because it will let us neatly define the loss function for the VAE, generate randomly sampled latent variables, achieve improved network generalization, **and** make our complete VAE network differentiable so that it can be trained via backpropagation. Quite powerful!

Let's define a function to implement the VAE sampling operation:

In [ ]:
"""Reparameterization trick by sampling from an isotropic unit Gaussian.
# Arguments
    args (tensor): mean and log of standard deviation of latent distribution (Q(z|X))
# Returns
    z (tensor): sampled latent vector
"""
def sampling(args):
    z_mean, z_logsigma = args
    batch = tf.shape(z_mean)[0]#z_mean.shape[0]
    dim = tf.shape(z_mean)[1]#z_mean.shape[1]

    # by default, random_normal has mean=0 and std=1.0
    epsilon = tf.random.normal(shape=(batch, dim))
    '''TODO: Define the reparameterization computation using the equation in the text above.'''
    z = z_mean + tf.exp(0.5 * z_logsigma) * epsilon
    return z

## 2.4 Debiasing variational autoencoder (DB-VAE) for facial detection

Now, we'll use the general idea behind the VAE architecture to build a model, termed a *debiasing variational autoencoder* or DB-VAE, to mitigate (potentially) unknown biases present within the training idea. We'll train our DB-VAE model on the facial detection task, run the debiasing operation during training, evaluate on the PPB dataset, and compare its accuracy to our original, biased CNN model.    

### The DB-VAE model

The key idea behind this debiasing approach is to use the latent variables learned via a VAE to adaptively re-sample the CelebA data during training. Specifically, we will alter the probability that a given image is used during training based on how often its latent features appear in the dataset. So, faces with rarer features (like dark skin, sunglasses, or hats) should become more likely to be sampled during training, while the sampling probability for faces with features that are over-represented in the training dataset should decrease (relative to uniform random sampling across the training data). 

A general schematic of the DB-VAE approach is shown here:

![DB-VAE](https://raw.githubusercontent.com/aamini/introtodeeplearning_labs/2019/lab2/img/DB-VAE.png)

Recall that we want to apply our DB-VAE to a *supervised classification* problem -- the facial detection task. Importantly, note how the encoder portion in the DB-VAE architecture also outputs a single supervised variable, $z_o$, corresponding to the class prediction -- face or not face. Usually, VAEs are not trained to output any supervised variables (such as a class prediction)! This is another key distinction between the DB-VAE and a traditional VAE. 

Keep in mind that we only want to learn the latent representation of *faces*, as that's what we're ultimately debiasing against, even though we are training a model on a binary classification problem. We'll need to ensure that, **for faces**, our DB-VAE model both learns a representation of the unsupervised latent variables, captured by the distribution $q_\phi(z|x)$, **and** outputs a supervised class prediction $z_o$, but that, **for negative examples**, it only outputs a class prediction $z_o$.

### Defining the DB-VAE loss function

This means we'll need to be a bit clever about the loss function for the DB-VAE. The form of the loss will depend on whether it's a face image or a non-face image that's being considered. 

For **face images**, our loss function will have two components:


1.   **VAE loss ($L_{VAE}$)**: consists of the latent loss and the reconstruction loss.
2.   **Classification loss ($L_y(y,\hat{y})$)**: standard cross-entropy loss for a binary classification problem. 

In contrast, for images of non-faces, our loss function is solely the classification loss. 

We can write a single expression for the loss by defining an indicator variable $\mathcal{I}_f$which reflects which training data are images of faces ($\mathcal{I}_f(x) = 1$ ) and which are images of non-faces ($\mathcal{I}_f(x) = 0$). Using this, we obtain:

$$L_{total} = L_y(y,\hat{y}) + \mathcal{I}_f(x)\Big[L_{VAE}\Big]$$

Let's write a function to define the DB-VAE loss function:



In [ ]:
# Loss function for DB-VAE
def debiasing_loss_function(x, x_pred, y, y_logit, mu, logsigma):

  '''TODO: call the relevant function that you created above to obtain the total VAE loss'''
  vae_loss = vae_loss_function(x, x_pred, mu, logsigma)
  '''TODO: define the classification loss for the binary classification task.'''
  # [images_per_batch, output_size=1]
  classification_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_logit)
  
  # Use the training data labels to create variable face_mask
  # This is I_f in the equation in the text above.
  face_mask = tf.cast(tf.equal(y, 1), tf.float32)
  
  '''TODO: define the DB-VAE total loss, based on the equation in the text above.
  Think about the dimensionality of your output and why tf.reduce_mean is being used'''
  total_loss = tf.reduce_mean(classification_loss + face_mask * vae_loss)
  
  return total_loss, classification_loss

### DB-VAE architecture

Now we're ready to define the DB-VAE architecture. First, let's define some key parameters for our model: the number of latent variables, the number of supervised outputs, and the starting number of filters for the first convolutional layer in the encoder.

In [ ]:
latent_dim = 100

To build the DB-VAE, we'll define each of the encoder and decoder networks separately, create and initialize the two models, and then construct the end-to-end VAE. We'll go through each of these steps in turn. 

In [ ]:
'''Define the encoder network for the DB-VAE'''
def make_face_encoder_network():
    Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
    BatchNormalization = tf.keras.layers.BatchNormalization
    Flatten = tf.keras.layers.Flatten
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

    inputs = tf.keras.layers.Input(shape=(64,64,3))
    
    hidden = Conv2D(filters=1*n_filters, kernel_size=[5,5],  strides=[2,2])(inputs)
    hidden = BatchNormalization()(hidden)
    hidden = Conv2D(filters=2*n_filters, kernel_size=[5,5],  strides=[2,2])(hidden)
    hidden = BatchNormalization()(hidden)
    hidden = Conv2D(filters=4*n_filters, kernel_size=[3,3],  strides=[2,2])(hidden)
    hidden = BatchNormalization()(hidden)
    hidden = Conv2D(filters=6*n_filters, kernel_size=[3,3],  strides=[1,1])(hidden)
    hidden = BatchNormalization()(hidden)

    hidden = Flatten(name='flatten')(hidden)
#     hidden = Dense(128)(hidden)
    
    '''Encoder outputs:
        y_logit: supervised class prediction
        z_mean: means in the latent space
        z_logsigma: standard deviations in the latent space'''
    y_logit = Dense(1, activation=None, name='y_logit')(hidden)
    z_mean = Dense(latent_dim, name='z_mean')(hidden)
    z_logsigma = Dense(latent_dim, name='z_logsigma')(hidden)

    # use reparameterization trick to sample from the latent space
    z = tf.keras.layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_logsigma])

    # define the outputs that the encoder model should return
    outputs = [y_logit, z_mean, z_logsigma, z]
    # finalize the encoder model
    encoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='encoder')

    # get the shape of the final convolutional output (right before the flatten)
    flatten_layer_idx = encoder.layers.index(encoder.get_layer('flatten'))
    pre_flatten_shape = encoder.layers[flatten_layer_idx-1].get_output_at(0).shape[1:]
    
    return encoder, inputs, outputs, pre_flatten_shape

Similarly, we can define the decoder network, which takes as input the sampled latent variables, runs them through a series of deconvolutional layers, and outputs a reconstruction of the original input image:

In [ ]:
'''Define the decoder network for the DB-VAE'''
def make_face_decoder_network(pre_flatten_shape):
  Conv2DTranspose = functools.partial(tf.keras.layers.Conv2DTranspose, padding='same', activation='relu')
  BatchNormalization = tf.keras.layers.BatchNormalization
  Flatten = tf.keras.layers.Flatten
  Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

  latent_inputs = tf.keras.layers.Input(shape=(latent_dim,))
  
#   hidden = Dense(128)(latent_inputs)
  hidden = Dense(tf.reduce_prod(pre_flatten_shape))(latent_inputs)
  hidden = tf.keras.layers.Reshape(pre_flatten_shape)(hidden)
  
  # series of deconvolutional layers with batch normalization
  hidden = Conv2DTranspose(filters=4*n_filters, kernel_size=[3,3],  strides=[1,1])(hidden)
  hidden = BatchNormalization()(hidden)
  hidden = Conv2DTranspose(filters=2*n_filters, kernel_size=[3,3],  strides=[2,2])(hidden)
  hidden = BatchNormalization()(hidden)
  hidden = Conv2DTranspose(filters=1*n_filters, kernel_size=[5,5],  strides=[2,2])(hidden)
  hidden = BatchNormalization()(hidden)
  
  x_hat = Conv2DTranspose(filters=3, kernel_size=[5,5], strides=[2,2])(hidden)

  # instantiate decoder model
  decoder = tf.keras.Model(inputs=latent_inputs, outputs=x_hat, name='decoder')
  return decoder


Now, call these functions to create the encoder and decoder!

In [ ]:
'''TODO: call the functions you defined above to create the encoder and decoder networks'''
encoder, inputs, ouputs, pre_flatten_shape = make_face_encoder_network()
decoder = make_face_decoder_network(pre_flatten_shape)

# initialize the models
encoder_output = encoder(inputs)
y_logit, z_mean, z_logsigma, z = encoder_output
reconstructed_inputs = decoder(z)

Finally we can construct our network end-to-end.

In [ ]:
# Construct the end to end vae
vae = tf.keras.Model(inputs, reconstructed_inputs)

Let's visualize the architecture of the encoder to get a more concrete understanding of this network,

In [ ]:
mdl.util.display_model(encoder)

As you can see, the encoder architecture is virtually identical to the CNN from earlier in this lab. Note the outputs of this model: `y_logit, z_mean, z_logsigma, z`. Think carefully about why each of these are outputted and their significance to the problem at hand.



### Adaptive resampling for automated debiasing with DB-VAE

So, how can we actually use DB-VAE to train a debiased facial detection classifier? Recall the DB-VAE architecture. As the input images are fed through the network, the encoder learns an estimate $\mathcal{Q}(z|X)$ of the latent space. We want to increase the relative frequency of rare data by increased sampling of under-represented regions of the latent space. We can approximate $\mathcal{Q}(z|X)$ using the frequency distributions of each of the learned latent variables, and then define the probability distribution of selecting a given datapoint $x$ based on this approximation. These probability distributions will be used during training to re-sample the data.

You'll write a function to execute this update of the sampling probabilities, and then call this function within the DB-VAE training loop to actually debias the model. 

First, we've defined a short helper function `get_latent_mu` that returns the latent variable means returned by the encoder after a batch of images is inputted to the network:

In [ ]:
# Function to return the means for an input image batch
def get_latent_mu(images, encoder, batch_size=1024):
    N = images.shape[0]
    mu = np.zeros((N, latent_dim))
    for start_ind in range(0, N, batch_size):
        end_ind = min(start_ind+batch_size, N+1)
        batch = images[start_ind:end_ind]
        batch = tf.convert_to_tensor(batch, dtype=tf.float32)/255.
        _, batch_mu, _, _ = encoder(batch)
        mu[start_ind:end_ind] = batch_mu
    return mu

Now, let's define the actual resampling algorithm `get_training_sample_probabilities`. Importantly note the argument `smoothing_fac`. This parameter tunes the degree of debiasing: for `smoothing_fac=0`, the re-sampled training set will tend towards falling uniformly over the latent space. 

In [ ]:
'''Function that recomputes the sampling probabilities for images within a batch
    based on how they distribute across the '''
def get_training_sample_probabilities(images, encoder, bins=10, smoothing_fac=0.0): 
    print("Recomputing the sampling probabilities")
    
    mu = get_latent_mu(images, encoder)
    # sampling probabilities for the images
    training_sample_p = np.zeros(mu.shape[0])
    
    # consider the distribution for each latent variable 
    for i in range(latent_dim):
      
        latent_distribution = mu[:,i]
        '''TODO: generate a histogram of the latent distribution'''
        hist_density, bin_edges = np.histogram(latent_distribution, density=True, bins=bins)

        # find which latent bin every data sample falls in 
        # https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.digitize.html
        bin_edges[0] = -float('inf')
        bin_edges[-1] = float('inf')
        bin_idx = np.digitize(latent_distribution, bin_edges)

        # smooth the density function (Eq. #4 in the DB-VAE paper)
        hist_smoothed_density = hist_density + smoothing_fac
        hist_smoothed_density = hist_smoothed_density / np.sum(hist_smoothed_density)

        p = 1.0/(hist_smoothed_density[bin_idx-1])
        
        # normalize all probabilities
        p = p / np.sum(p)
        
        # update sampling probabilities 
        training_sample_p = np.maximum(p, training_sample_p)
        
    # final normalization
    training_sample_p /= np.sum(training_sample_p)

    return training_sample_p

Now that we've defined the resampling update, we can train our DB-VAE model on the CelebA/ImageNet training data, and run the above operation to re-weight the importance of particular data points as we train the model. Remember again that we only want to debias for features relevant to *faces*, not the set of negative examples.

Complete the code block below to execute the training loop!

In [ ]:
loss_history = []
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

enable_debiasing = True
all_faces = loader.get_all_train_faces() # parameter from data loader

for epoch in range(num_epochs):
  
  # progress message and bar
#   custom_msg = util.custom_progress_text("Epoch: %(epoch).0f   Iter: %(idx).0f   Class Loss: %(class_loss)2.2f   Loss: %(loss)2.2f")
#   bar = util.create_progress_bar(custom_msg)

  p_faces = None
  if enable_debiasing: 
      # Recompute data sampling proabilities if debiasing is enabled
      '''TODO: call the get_training_sample_probabilities function
      to recompute the sampling probabilities when debiasing is enabled'''
      p_faces = get_training_sample_probabilities(all_faces, encoder, 10, 0.0)
  
  for idx in range(loader.get_train_steps_per_epoch(batch_size)):
    # load a batch of data
    (x, y) = loader.get_batch(batch_size, p_pos=p_faces)
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
  
    # define GradientTape for automatic differentiation
    with tf.GradientTape() as tape:
      y_logit, mu, logsigma, z = encoder(x)
      x_hat = decoder(z)
      '''TODO: call the debiasing_loss_function to compute the loss'''
      loss, class_loss = debiasing_loss_function(x, x_hat, y, y_logit, mu, logsigma)
    
    '''TODO: use the GradientTape.gradient method to compute the gradients
    Hint: compute the gradient of the 'loss' with respect to the variables in the VAE
    Hint: recall how this is done in the standard CNN training code block'''
    grads = tape.gradient(loss, vae.variables)
    # apply gradients to variables
    optimizer.apply_gradients(zip(grads, vae.variables))

    # track the losses
    class_loss_value = class_loss.numpy().mean()
    loss_value = loss.numpy().mean()
    loss_history.append((class_loss_value, loss_value))
#     custom_msg.update_mapping(epoch=epoch, idx=idx, loss=loss_value, class_loss=class_loss_value)
    
    # plot the progress every 100 steps
#     if idx%100 == 0: 
#       mdl.util.plot_sample(x,y,vae)

Wonderful! Now we should have a trained and (hopefully!) debiased facial classification model, ready for evaluation!

In [ ]:
len(vae(x))

## 2.4 Evaluation on Pilot Parliaments Benchmark (PPB) Dataset

Finally let's test our DB-VAE model on the[ PPB dataset](http://proceedings.mlr.press/v81/buolamwini18a/buolamwini18a.pdf). 

We'll evaluate both the overall accuracy of the DB-VAE as well as its accuracy on each the "Dark Male", "Dark Female", "Light Male", and "Light Female" demographics, and compare the performance of this debiased model against the biased CNN from earlier in the lab. 

Here are some example images from the PPB dataset.
![PPB Example Images](https://raw.githubusercontent.com/aamini/introtodeeplearning_labs/2019/lab2/img/PPB%20faces.png)

To assess performance, we'll measure the classification accuracy of each model, which we define as the fraction of PPB faces detected. By comparing the accuracy of a model without debiasing and our DB-VAE model, we can get a sense of how effectively we were able to debias against features like skin tone and gender.

Let's evaluate our debiased model on the PPB test dataset.

In [ ]:
# Evaluate on PPB dataset (takes ~4 minutes)
accuracy_debiased = []
for skin_color in ['lighter', 'darker']:
  for gender in ['male', 'female']:
    accuracy_debiased.append( ppb.evaluate([encoder], gender, skin_color, output_idx=0, from_logit=True)[0] )
    print()
    print("{} {}: {}".format(gender, skin_color, accuracy_debiased[-1]))

We can calculate the accuracies of our model on the whole PPB dataset as well as across the four demographics proposed and visualize our results comparing to the standard, biased CNN.


In [ ]:
bar_width = 0.3
plt.bar(np.arange(4), standard_cnn_accuracy, width=bar_width)
plt.bar(np.arange(4)+bar_width, accuracy_debiased, width=bar_width)
plt.legend(('Standard Classifier','Debiased Classifier (DB-VAE)'))
plt.xticks(np.arange(4), ('LM', 'LF', 'DM', 'DF'))
plt.ylim(np.min([standard_cnn_accuracy,accuracy_debiased])-0.01,np.max([standard_cnn_accuracy,accuracy_debiased])+0.01)
plt.ylabel('Accuracy')

## 2.5 Conclusion 

We encourage you to think about and maybe even address some questions raised by the approach and results outlined here:

*  How does the accuracy of the DB-VAE across the four demographics compare to that of the standard CNN? Do you find this result surprising in any way?
*  In which applications (either related to facial detection or not!) would debiasing in this way be desired? Are there applications where you may not want to debias your model? 
* Do you think it should be necessary for companies to demonstrate that their models, particularly in the context of tasks like facial detection, are not biased? If so, do you have thoughts on how this could be standardized and implemented?
* Do you have ideas for other ways to address issues of bias, particularly in terms of the training data?

Hopefully this lab has shed some light on a few concepts, from vision based tasks, to VAEs, to algorithmic bias. We like to think it has, but we're biased ;). 

![Faces](https://media1.tenor.com/images/44e1f590924eca94fe86067a4cf44c72/tenor.gif?itemid=3394328)